# This is going to take in the fasta file and use the information in the header to write a few different files.

#### First we have our imports

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from collections import Counter
import csv

#### This is a little file opener and appender, we make a few lists using the headers in the fasta file to append the desired information into each list.

In [ ]:
header_time = []
header_size = []
header_vdj = []
header_seq = []
for record in SeqIO.parse('full_size-30.fasta', 'fasta'):
    #print(record.id)
    header_time.append(record.id.split('_')[1])
    header_size.append(record.id.split('_')[2])
    header_vdj.append(record.id.split('_')[3])
    header_seq.append(record.id.split('_')[4])
#print(len(header_time))
#print(len(header_size))
#print(len(header_vdj))
#print(header_time)

#### The code works by grabbing the correct info and appending it to the desired list, I am not sure if this data is actually the v, d, and j region tho..

In [ ]:
v_region = [] 
d_region = []
j_region = []
for i in header_vdj:
    v_region.append(i.split(',')[0])
    d_region.append(i.split(',')[1])
    j_region.append(i.split(',')[2])

#### This is a sanity check just to show that everything is accounted for 

In [ ]:
print(len(v_region))
print(len(d_region))
print(len(j_region))

#### This is a cool little block that uses the Counter function to see how many unique regions there are

In [ ]:
uni_v = Counter(v_region)
uni_d = Counter(d_region)
uni_j = Counter(j_region)

#### This is really interesting... I need to make sure I understand the data to see if I really have these regions. 

In [ ]:
print(len(uni_v))
print(len(uni_d))
print(len(uni_j))
print(len(header_time))

In [ ]:
lines = zip(header_time, v_region)

#### We are going to take our lists and write a nice csv file to visualize it. 

In [ ]:
with open('try.csv','w') as out:
    out.write("{},{},{}\n".format( str('#'), str('tp'), str('v')))
    for i,j in enumerate(lines):
        out.write("{},{},{}\n".format( i, j[0], j[1]))

#### ok this works and writes a nice csv. lets munge that data around with pandas to get what we want

#### Now we will put the VDJ notation on hold and focus more on the actual sequences
This step is taking the sequences in each header and translating it to amino acids.

In [ ]:
temp = []
for i in header_seq:
    temp.append(Seq(i))

#### We have to use this logic loop because there are some sequences in the headers that contain characters that throw an error. We take care of those sequences that throw errors, although we dont do anything with it down stream yet.

In [ ]:
head_seq_trans = []
weird_seq = []
for i in temp:
    if '-' in i:
        head_seq_trans.append('bad char in seq')
        weird_seq.append(i)
    else:
        head_seq_trans.append(i.translate())
    

#### This is a neat block that will take the v_region and append it to the end of each element in in head_seq_trans. So, now at the end of each translated sequence you will have its corresponding v_region

In [ ]:
seq_v = [head_seq_trans[i] + '_' + v_region[i] for i in range(len(head_seq_trans))]

In [ ]:
for i in seq_v[0:1]:
    print(i)

In [ ]:
lines = zip(header_time, seq_v)

#### Lastly, we will write our csv file that contains the desired information

In [ ]:
with open('seq.csv','w') as out:
    out.write("{},{},{}\n".format( str('#'), str('tp'), str('seq_v')))
    for i,j in enumerate(lines):
        out.write("{},{},{}\n".format( i, j[0], j[1]))

In [ ]:
len(header_seq)

#### This little block is another sanity check, however we see that there are 1062 header sequences total and when we translate the sequences, there are 927 unique, translated seqs (might want to do something with these later).

In [ ]:
uni_trans_seq = Counter(head_seq_trans)

In [ ]:
len(uni_trans_seq)